In [179]:
from bs4 import BeautifulSoup
import requests
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver import ChromeOptions
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [180]:
def beautifulsoup_extractor(URL):
    page = requests.get(URL)
    soup = BeautifulSoup(page.text, "html")

In [181]:
def flatten_and_append(data, new_list=[]):
  
  for item in data:
    if isinstance(item, list):
      new_list = flatten_and_append(item, new_list)
    else:
      new_list.append(item)

  return new_list

In [182]:
def selenium_extractor(URL):
    
    options = Options()
    options.add_argument("--headless") 
    driver = webdriver.Chrome(options=options)
    driver.get(URL)
    element_starters = driver.find_elements(By.TAG_NAME, "tbody")

    player_stats_starters = element_starters[1].text

    element_bench = driver.find_elements(By.CLASS_NAME, "bench")
    player_stats_bench = element_bench[0].text
    driver.quit()

In [183]:
beautifulsoup_extractor("https://fibalivestats.dcd.shared.geniussports.com/u/ABE/2310604/bs.html")

In [184]:
boxscore = soup.find_all("table")[1]
column_names = []
headers = boxscore.find("thead").findAll("th")

for th in headers:
    column_names.append(th.text.strip())
column_names = column_names[:24]

In [185]:
selenium_extractor("https://fibalivestats.dcd.shared.geniussports.com/u/ABE/2310604/bs.html")

In [186]:
player_stats_splitted = player_stats_starters.split("\n")

for i in range(0, len(player_stats_splitted), 3):
  player_stats_splitted[i+2] = player_stats_splitted[i+2].split(" ")
    

flattened_list = flatten_and_append(player_stats_splitted)
starters = {f"starter_{i + 1}": flattened_list[i:i+24] for i in range(0, len(flattened_list), 24)}


In [187]:
import pandas as pd
df = pd.DataFrame(columns= column_names)

for key, value in starters.items():
    df.loc[len(df)] = value

In [188]:
df

,No.,Jugador,POS,MIN,Pt,TC,Ti%,2P,2P%,3P,...,RD,RT,AS,PER,REC,Tap,TapRe,FP,FR,+-Pt
0,1,J. Sanchez,G,32:23,21,7-11,63,5-7,71,2-4,...,2,2,1,1,3,0,0,2,6,19
1,10,J. Puente,PG,23:51,6,1-3,33,0-2,0,1-1,...,2,2,1,3,0,0,0,5,2,22
2,11,J. Sanchez,G,21:10,7,3-6,50,2-5,40,1-1,...,0,1,1,4,2,0,0,2,2,8
3,13,R. Flores,G,28:00,10,2-10,20,2-9,22,0-1,...,6,7,0,3,4,0,1,2,4,22
4,37,R. De La Mora,F,22:13,10,2-4,50,2-2,100,0-2,...,7,9,1,3,0,0,0,4,5,14
